# Non-Gaussian Measurements Tutorial 2

## Overview

- This tutorial introduces non-Gaussian estimation via a factor graph solution.
- Factor graphs should be thought of fully probabilisitic interactions between variables, as modelled by the factors.
- This tutorial shows how measurement statistics do not have be unimodal bell curve (i.e. normal/Gaussian) and thereby introduce non-Gaussian behavior.
- This example will use only continuous stochastic variables with slightly more diverse measurement distribution.
- This tutorial is one dimension with only a few variables and factors, in order to get comfortable working with multi-modal beliefs.
- This example is showing one of four mechanisms by which non-Gaussian behavior can get introduced into a factor graph solution, see other tutorials for other mechanisms.
- This tutorial illustrates how algebraic relations (i.e. residual functions) between multiple stochastic variables are calculated, as well as the final posterior belief estimate, from several pieces of information.
- The tutorial implicitly shows a multi-modal uncertainty can be introduced from non-Gaussian measurements, and then transmitted through the system.
- The tutorial also illustrates consensus through an additional piece of information, which reduces all stochastic variable marginal beliefs to unimodal only beliefs.
- The ambiguous measurement data example shown in this tutorial can readily be incorporated in other use cases.
- Lastly, this tutorial will also show how graph-based automatic initialization of variables is achieved with the Caesar.jl solver.

### Signatures Used

`ContinuousScalar`, `Prior`, `LinearRelative`, `Mixture`, `Normal`, `plotKDE`, `plotDFG`, `isInitialized`, `initAll!`, `solveGraph!`

## Ambiguous Data Example

The application of this tutorial is presented in abstract from which the user is free to imagine any system of relationships:  For example, a robot driving in a one dimensional world; or a time traveler making uncertain jumps 
forwards and backwards in time.

To start, the two major mathematical packages are brought into scope.

In [1]:
# import Pkg; Pkg.add(Pkg.PackageSpec(name="IncrementalInference", version="0.27"))
using IncrementalInference

┌ Info: FunctionalStateMachine.jl is adding Graphs.jl related tools (Visualization).
└ @ FunctionalStateMachine /home/samc/.julia/packages/FunctionalStateMachine/2JZFG/src/StateMachineAnimation.jl:1
┌ Info: IncrementalInference.jl is loading tools related to InteractiveUtils.jl.
└ @ IncrementalInference /home/samc/.julia/packages/IncrementalInference/zwqKk/src/RequireInteractiveUtils.jl:2


 
## Starting a 1D Factor Graph

### Variable, `ContinuousScalar`

The next step is to describe the inference problem with a graphical model with any of the existing concrete types that inherit from  `<: AbstractDFG`.
The first step is to create an empty factor graph object and start populating it with variable nodes.
The variable nodes are identified by `Symbol`s, namely `:x0, :x1, :x2, :x3`.


In [2]:
# Start with an empty factor graph
fg = initfg()

# add the first node
addVariable!(fg, :x0, ContinuousScalar)

DFGVariable{Position...}
  manifold:   TranslationGroup(1; field = ℝ)
  timestamp:  2022-03-23T07:08:48.944-05:00
   nstime:    0 nanoseconds
  label:      x0
  solvable:   1
  tags:       Set([:VARIABLE])
  # VND solveKeys=    (1)
  # initialized:      (true=0,false=1)
  # marginalized:     (true=0,false=1)
    :default <-- VariableNodeData
      initilized:        false
      marginalized:      false
      size bel. samples: (100,)
      kde bandwidths:    0.0
     VNDs: [:default]
  # PPE solveKeys=    (0)
  VariableType: Position{1}


### Prior Factor, (Euclidean(1))

Factor graphs are bipartite graphs with `factors` that act as mathematical structure between interacting `variables`.
After adding node `:x0`, a singleton factor of type `Prior` (which was defined by the user earlier) is 'connected to' variable node `:x0`.
This unary factor is taken as a `Distributions.Normal` distribution with zero mean and a standard devitation of `1`.


In [3]:
# this is unary (prior) factor and does not immediately trigger autoinit of :x0.
addFactor!(fg, [:x0], Prior(Normal(0,1)))

[ Info: try doautoinit! of x0


DFGFactor{CommonConvWrapper{Prior...}}
  timestamp:     2022-03-23T07:09:15.297-05:00
   nstime:       0 nanoseconds
  label:         x0f1
  solvable:      1
  VariableOrder: [:x0]
  multihypo:     Float64[]
  nullhypo:      0.0
  tags:          Set([:FACTOR])
  FactorType: Prior{Normal{Float64}}
Z:
Normal{Float64}(μ=0.0, σ=1.0)


**Note**, this example is using just `Prior`, which is a simplification for the sake of this tutorial.  Look out for on-manifold prior factors in other code examples.

### Visualizing Graph Topology

In [ ]:
using GraphPlot
DFG.plotDFG(fg)


The topological graph plot above shows the two node factor graph, one variable and one prior factor.  
This example uses graph-based automatic variable initialization which was discussed in ICRA 2022 Tutorial 1.

### Next Variable and Relative Factor

Now let's add a second variable `:x1`, and connect it to `:x0` with a `LinearRelative` factor.

In [4]:
addVariable!(fg, :x1, ContinuousScalar)
# P(Z | :x1 - :x0 ) where Z ~ Normal(10,1)
addFactor!(fg, [:x0, :x1], LinearRelative(Normal(10.0,1)))

[ Info: try doautoinit! of x0
[ Info: init with useinitfct [:x0f1]


[ Info: do init of x0
┌ Warning: updateVariableSolverData with solveKey parameter might change in the future, see DFG #565. Future warnings are suppressed
└ @ DistributedFactorGraphs /home/samc/.julia/packages/DistributedFactorGraphs/ttnFL/src/services/DFGVariable.jl:658


[ Info: try doautoinit! of x1


DFGFactor{CommonConvWrapper{LinearRelative...}}
  timestamp:     2022-03-23T07:09:30.122-05:00
   nstime:       0 nanoseconds
  label:         x0x1f1
  solvable:      1
  VariableOrder: [:x0, :x1]
  multihypo:     Float64[]
  nullhypo:      0.0
  tags:          Set([:FACTOR])
  FactorType: LinearRelative{1, Normal{Float64}}
Z:
Normal{Float64}(μ=10.0, σ=1.0)


### Visualizing the Variable Probability Belief

The `RoMEPlotting.jl` package allows visualization (plotting) of the belief state over any of the variable nodes.
Remember the first time executions are slow given required code compilation, and that future versions of these package will use more precompilation to reduce first execution running cost.

In [ ]:
using RoMEPlotting

plotKDE(fg, :x0)


By forcing the initialization of `:x1` and plotting its belief estimate,

In [ ]:
initAll!(fg)
plotKDE(fg, [:x0, :x1])

the predicted influence of the `P(Z| X1 - X0) = LinearRelative(Normal(10, 1))` is shown by the red trace.
The red trace (predicted belief of `:x1`) is noting more than the approximated convolution of the current marginal belief of `:x0` with the conditional belief described by `P(Z | X1 - X0)`.


### Mixture Distribution on Next Relative Factor

Another `ContinuousScalar` variable `:x2` is 'connected' to `:x1` through a more complicated `MixtureRelative` likelihood function.

In [5]:
addVariable!(fg, :x2, ContinuousScalar)
mmo = Mixture(LinearRelative, 
              (hypo1=Rayleigh(3), hypo2=Uniform(30,55)), 
              [0.4; 0.6])
addFactor!(fg, [:x1, :x2], mmo)

[ Info: try doautoinit! of x1
[ Info: init with useinitfct [:x0x1f1]
[ Info: do init of x1
[ Info: try doautoinit! of x2


DFGFactor{CommonConvWrapper{Mixture...}}
  timestamp:     2022-03-23T07:09:55.020-05:00
   nstime:       0 nanoseconds
  label:         x1x2f1
  solvable:      1
  VariableOrder: [:x1, :x2]
  multihypo:     Float64[]
  nullhypo:      0.0
  tags:          Set([:FACTOR])
  FactorType: Mixture{2, LinearRelative, (:hypo1, :hypo2), Tuple{Rayleigh{Float64}, Uniform{Float64}}}
mechanics:
LinearRelative{1, FullNormal}(FullNormal(
dim: 1
μ: [0.0]
Σ: [1.0]
)
)
components:
(hypo1 = Rayleigh{Float64}(σ=3.0), hypo2 = Uniform{Float64}(a=30.0, b=55.0))
diversity:
Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.4, 0.6])
dims:
1
labels:
[2]


In [13]:
using DistributedFactorGraphs
pack = packFactor(fg, getFactor(fg, :x1x2f1))
using JSON
@info JSON.json(pack)

┌ Info: {"label":"x1x2f1","_version":"0.18.1","_variableOrderSymbols":["x1","x2"],"data":{"eliminated":false,"potentialused":false,"edgeIDs":[],"fnc":{"N":2,"F_":"PackedLinearRelative","S":["hypo1","hypo2"],"components":[{"_type":"IncrementalInference.PackedRayleigh","sigma":3.0},{"_type":"IncrementalInference.PackedUniform","a":30.0,"b":55.0,"PackedSamplableTypeJSON":"IncrementalInference.PackedUniform"}],"diversity":{"_type":"IncrementalInference.PackedCategorical","p":[0.4,0.6]}},"multihypo":[],"certainhypo":[1,2],"nullhypo":0.0,"solveInProgress":0,"inflation":5.0},"tags":["FACTOR"],"timestamp":"2022-03-23T07:09:55.020-05:00","nstime":"0","fnctype":"Mixture","solvable":1}
└ @ Main /home/samc/Ganbatte/BinderNotebooks/julia/caesar-api/icra-2-nongaussian.ipynb:4



The `mmo` variable illustrates how a near arbitrary mixture probability distribution can be used as a conditional relationship between variable nodes in the factor graph.
In this case, a 40%/60% balance of a Rayleigh and truncated Uniform distribution which acts as a multi-modal conditional belief.
Interpret carefully what a conditional belief of this nature actually means.

Following the tutorial's practical example frameworks (robot navigation or time travel), this multi-modal belief implies that moving from one of the probable locations in `:x1` to a location in `:x2` by some processes defined by `mmo=P(Z | X2, X1)` is uncertain to the same 40%/60% ratio.
In practical terms, collapsing (through observation of an event) the probabilistic likelihoods of the transition from `:x1` to `:x2` may result in the `:x2` location being at either 15-20, or 40-65-ish units.
The predicted belief over `:x2` is illustrated by plotting the predicted belief (green trace), after forcing initialization.

In [ ]:
initAll!(fg)
plotKDE(fg, [:x0, :x1, :x2])


## Adding Variable `x3`

Adding one more variable `:x3` through another `LinearRelative(Normal(-50,1))`

In [6]:
addVariable!(fg, :x3, ContinuousScalar)
addFactor!(fg, [:x2, :x3], LinearRelative(Normal(-50, 1)))

[ Info: try doautoinit! of x2
[ Info: init with useinitfct [:x1x2f1]
[ Info: do init of x2
[ Info: try doautoinit! of x3


DFGFactor{CommonConvWrapper{LinearRelative...}}
  timestamp:     2022-03-23T07:10:50.911-05:00
   nstime:       0 nanoseconds
  label:         x2x3f1
  solvable:      1
  VariableOrder: [:x2, :x3]
  multihypo:     Float64[]
  nullhypo:      0.0
  tags:          Set([:FACTOR])
  FactorType: LinearRelative{1, Normal{Float64}}
Z:
Normal{Float64}(μ=-50.0, σ=1.0)


expands the factor graph to to four variables and four factors.
This part of the tutorial shows how a unimodal likelihood (conditional belief) can transmit the bimodal belief currently contained in `:x2`.

In [ ]:
initAll!(fg)
plotKDE(fg, [:x0, :x1, :x2, :x3])

Notice the blue trace (`:x3`) is a shifted and slightly spread out version of the initialized belief on `:x2`, through the convolution with the conditional belief `P(Z | X2, X3)`.

### The Last Factor


Global inference over the entire factor graph has still not occurred, and will at this stage produce roughly similar results to the predicted beliefs shown above.
Only by introducing more information into the factor graph can inference extract more precise marginal belief estimates for each of the variables.
A final piece of information added to this graph is a factor directly relating `:x3` with `:x0`.

In [7]:
addFactor!(fg, [:x3, :x0], LinearRelative(Normal(40, 1)))

[ Info: try doautoinit! of x3
[ Info: init with useinitfct [:x2x3f1, :x3x0f1]
[ Info: do init of x3


DFGFactor{CommonConvWrapper{LinearRelative...}}
  timestamp:     2022-03-23T07:11:01.489-05:00
   nstime:       0 nanoseconds
  label:         x3x0f1
  solvable:      1
  VariableOrder: [:x3, :x0]
  multihypo:     Float64[]
  nullhypo:      0.0
  tags:          Set([:FACTOR])
  FactorType: LinearRelative{1, Normal{Float64}}
Z:
Normal{Float64}(μ=40.0, σ=1.0)


Pay close attention to what this last factor means in terms of the probability density traces shown in the previous figure.
The blue trace for `:x3` has two major modes, one that overlaps with `:x0, :x1` near 0 and a second mode further to the left at -40.
The last factor introduces a shift `LinearRelative(Normal(40,1))` which essentially aligns the left most mode of `:x3` back onto `:x0`.

This last factor forces a mode selection through consensus.
By doing global inference, the new information obtained in `:x3` will be equally propagated to `:x2` where only one of the two modes will remain.

## Solve the Graph


Global inference is achieved with local computation using two function calls, as follows.


In [ ]:
tree = solveGraph!(fg)

# and visualization
plotKDE(fg, [:x0, :x1, :x2, :x3])

The resulting posterior marginal beliefs over all the system variables are:

## Conclusion

It is import to note that although this tutorial ends with all marginal beliefs having near Gaussian shape and are unimodal, that the package supports multi-modal belief estimates during both the prediction and global inference processes.
In fact, many of the same underlying inference functions are involved with the automatic initialization process and the global multi-modal iSAM inference procedure.
This concludes the ContinuousScalar tutorial

In [10]:
using DistributedFactorGraphs
vs = ls(fg)
for v in vs
  @info packVariable(fg, getVariable(fg, v))
end
#fs = lsf(fg)

┌ Info: Dict{String, Any}("label" => "x0", "dataEntry" => "{}", "nstime" => "0", "dataEntryType" => "{}", "ppeDict" => "{\"default\":{\"solveKey\":\"default\",\"suggested\":[0.002496014214205152],\"max\":[0.23310151135987967],\"mean\":[0.002496014214205152],\"lastUpdatedTimestamp\":\"2022-03-23T07:09:40.844\"}}", "solverDataDict" => "{\"default\":{\"vecval\":[1.5306267503501014,-0.2294976819052366,-0.313603214217259,0.4701066690583663,-0.10635476760697264,0.9354501620816313,1.1793238836401907,0.30382718575092615,0.7794806615864662,0.6433402141724786,1.8721294522158753,0.7899032446314936,-0.05404181757948353,1.4804144797831782,-0.10368283049764353,-0.7957685219413916,0.5656287227460458,-1.0092548336698153,-2.452379720431688,1.6299555436963542,-0.7770595397579406,-0.20412860301926436,1.0479848657023905,0.4225892701015142,-1.0052926768601078,-0.2676192557809827,1.2001794514483206,0.1288756892440473,0.6560221708749188,-0.2370100577665365,-1.7282323271736986,2.479952378793093,-1.33047777770

┌ Info: Dict{String, Any}("label" => "x3", "dataEntry" => "{}", "nstime" => "0", "dataEntryType" => "{}", "ppeDict" => "{\"default\":{\"solveKey\":\"default\",\"suggested\":[-39.50502665934523],\"max\":[-39.58882115187409],\"mean\":[-39.50502665934523],\"lastUpdatedTimestamp\":\"2022-03-23T07:11:07.482\"}}", "solverDataDict" => "{\"default\":{\"vecval\":[-39.53963537863787,-40.74636407621055,-39.59750104980447,-41.67294924175446,-39.081137162035056,-37.69868686863006,-39.23913599632294,-38.438570405255604,-41.58005404868901,-37.55591492039292,-40.059671266335314,-38.49546414945179,-39.98166437817355,-40.45630373219981,-38.7728549725746,-40.35354195153519,-39.50702939447998,-40.006681774365006,-36.8523733590635,-35.282045122187185,-38.18568815212143,-38.75406111443712,-39.106612651065475,-40.527107671090285,-40.25203075221147,-39.03144424049256,-41.265141391514824,-41.15175610418193,-39.14816035585719,-41.09483841039708,-37.4600689879028,-36.23738897730057,-41.61796561322531,-40.1239064